## An Improved Chemical Reaction Optimization Algorithm for Solving the Shortest Common Supersequence Problem

In [23]:
#Library 
import random
import math
import copy

In [6]:
class Test(CRO):
    """
    This is a derived class to find the minimum value of sin(x) + cos(y).
    In real terms, you should implement this class according to your real problem.
    Note that every copy operation is deep-copy.
    """
    def __init__(self, structure):
        CRO.__init__(self, structure)

    def wall(self, m):
        new = copy.deepcopy(m)
        new.structure[0], new.structure[1] = new.structure[1], new.structure[0]
        return new

    def dec(self, m):
        new1 = copy.deepcopy(m)
        new2 = copy.deepcopy(m)
        new1.structure[0] += random.random()
        new2.structure[1] += random.random()
        return [new1, new2]

    def inter(self, m1, m2):
        new1 = copy.deepcopy(m1)
        new2 = copy.deepcopy(m2)
        new1.structure[0] = m2.structure[0]
        new1.structure[1] = m1.structure[1]
        new2.structure[0] = m1.structure[0]
        new2.structure[1] = m2.structure[1]
        return [new1, new2]

    def syn(self, m1, m2):
        new = copy.deepcopy(m1)
        if random.random() < 0.5:
            new.structure[0] = m2.structure[0]
        else:
            new.structure[1] = m2.structure[1]
        return new

    def fit_func(self, m):
            return math.sin(m.structure[0]) + math.cos(m.structure[1])

### Parameters 
- Popsize: Set of all feasible solutions
- PE (potential energy): The objective function value related to a corresponding molecule
- KE (kinetic energy): Numerical value of the amount of tolerance to accept a worse solution
- NumHit: Number of collisions by a molecule
- KELossRate: Percentage of the upper limit of KE reduction
- MoleColl: Threshold to determine the type of chemical reaction: uni-molecule or inter-molecule
- Initial KE: Initial value of the kinetic energy assigned to each molecule in the initialization stage
- α, β :Threshold values for the intensification and diversification
- MinStruct: The molecule structure that has minimum potential
- MinHit: The number of hits when a molecule has MinStruct

## IMCRO 
- Algorithm 1: initialization, iteration,
and the finalization

### 1. Initialization
- supersequence_generate
- population_generation
- encoding_population

### 2. Iteration:
- reaction and repair
- In reaction, four main operators:
  * on-wall ineffective collision, 
  * decomposition,
  * inter-molecular
  * ineffective collision 
  * synthesis. 
- They fall into two categories: uni-molecule
reaction(1 molecule) and inter-molecule reaction(more than 1 molecule)

- At the beginning of the main iteration in Algorithm 1, a parameter t is randomly generated. It determines which type of reaction, e.g. uni-molecule or inter-molecule reaction, will be triggered. If t > MoleColl, uni-molecule reactions are triggered; otherwise, inter-molecule reactions are trigged. In the uni-molecule reaction, the parameter α determines which type of reaction occurs. If (NumHit − MinHit) > α, decomposition occurs; otherwise, on-wall ineffective collision occurs. In the same way, in the inter-molecule reaction, the
parameter β determines the type of reaction. If KE ≤ β, synthesis occurs; otherwise, inter-molecular ineffective collision occurs

In [7]:
class Molecule:
    def __init__(self, structure):
        self.pe = 0
        self.ke = 0
        self.opt = 9999999
        self.num_of_hits = 0
        self.structure = copy.deepcopy(structure)

    def update(self):
        """
        If this molecule has a lower energy, reset num_of_hits.
        """
        if self.pe < self.opt:
            self.opt = self.pe
            self.num_of_hits = 0
    def __str__(self):
        reVal = ''
        for i in self.structure:
            reVal+=str(i)+" "
        return reVal

class CRO:
    """
    A base class represent the CRO algorithm.
    You should inherit from this class.
    """
    optimal = None  # A Molecule object represent the optimal solution
    cnt = 0
    limit = 1000
    KELossRate = 0.2
    mole_coll = 0.2
    alpha = 100
    beta = 100
    buffer = 0
    init_ke = 100
    pop = []

    def __init__(self, structure):
        """
        * fit_func: Object function
        * structure: Initial solution list [s1, s2, ..., sn]
        """
        for s in structure:
            self.pop.append(Molecule(s))

        for mol in self.pop:
            # You should implement this function in your derived class
            mol.pe = self.fit_func(mol)
            mol.ke = self.init_ke
            mol.update()
            if self.optimal is None:
                self.optimal = copy.deepcopy(mol)
            elif mol.pe < self.optimal.pe:
                self.optimal = copy.deepcopy(mol)

    def run(self):
        while self.cnt < self.limit:
            self.cnt += 1
            if random.random() > self.mole_coll or len(self.pop) < 2:
                m = random.choice(self.pop)
                if m.num_of_hits > self.alpha:
                    self.decomposition(m)
                else:
                    self.on_wall(m)
            else:
                m1, m2 = random.sample(self.pop, 2)
                if m1.ke <= self.beta and m2.ke <= self.beta:
                    self.synthesis(m1, m2)
                else:
                    self.interaction(m1, m2)

    def update(self, m):
        """
        If m is the current optimal solution, save it to the optimal.
        """
        if m.pe < self.optimal.pe:
            self.optimal = copy.deepcopy(m)

    def decomposition(self, m):
        m.num_of_hits += 1

        # You should implement this function in your derived class
        new1, new2 = self.dec(m)
        new1.pe = self.fit_func(new1)
        new2.pe = self.fit_func(new2)
        tmp = m.pe + m.ke - new1.pe - new2.pe
        if tmp >= 0 or tmp + self.buffer >= 0:
            if tmp >= 0:
                q = random.random()
                new1.ke = tmp * q
                new2.ke = tmp * (1 - q)
            else:
                new1.ke = (tmp + self.buffer) * random.random() * random.random()
                new2.ke = (tmp + self.buffer - new1.ke) * random.random() * random.random()
                self.buffer = self.buffer + tmp - new1.ke - new2.ke
            new1.update()
            new2.update()
            self.pop.remove(m)
            self.pop.append(new1)
            self.pop.append(new2)
            self.update(new1)
            self.update(new2)

    def on_wall(self, m):
        m.num_of_hits += 1

        # You should implement this function in your derived class
        new = self.wall(m)
        new.pe = self.fit_func(new)
        tmp = m.pe + m.ke - new.pe
        if tmp >= 0:
            q = random.uniform(self.KELossRate, 1)
            new.ke = tmp * q
            new.update()
            self.buffer += tmp * (1 - q)
            self.pop.remove(m)
            self.pop.append(new)
            self.update(new)

    def interaction(self, m1, m2):
        m1.num_of_hits += 1
        m2.num_of_hits += 1

        # You should implement this function in your derived class
        new1, new2 = self.inter(m1, m2)
        new1.pe = self.fit_func(new1)
        new2.pe = self.fit_func(new2)
        tmp = m1.pe + m2.pe + m1.ke + m2.ke - new1.pe - new2.pe
        if tmp >= 0:
            q = random.random()
            new1.ke = tmp * q
            new2.ke = tmp * (1 - q)
            new1.update()
            new2.update()
            self.pop.remove(m1)
            self.pop.remove(m2)
            self.pop.append(new1)
            self.pop.append(new2)
            self.update(new1)
            self.update(new2)

    def synthesis(self, m1, m2):
        m1.num_of_hits += 1
        m2.num_of_hits += 1

        # You should implement this function in your derived class
        new = self.syn(m1, m2)
        new.pe = self.fit_func(new)
        tmp = m1.pe + m2.pe + m1.ke + m2.ke - new.pe
        if tmp >= 0:
            new.ke = tmp
            new.update()
            self.pop.remove(m1)
            self.pop.remove(m2)
            self.pop.append(new)
            self.update(new)

In [8]:
s = [[random.random(), random.random()] for i in range(10)]
print(s)
cro = Test(s)
cro.run()
print(cro.optimal.pe)

[[0.5357323770055559, 0.12347770609994357], [0.5086136930362755, 0.7916754198490201], [0.13454069036237892, 0.7244754118996637], [0.786439465272917, 0.26953456017904087], [0.533805237706414, 0.00989132435628648], [0.40380196214070596, 0.8481005121439357], [0.2730049177080778, 0.10669256617309486], [0.6413206573684568, 0.8661388436232256], [0.6243555718525715, 0.6010428322497526], [0.20275953724641582, 0.4382175202537315]]
-1.9885070682736965


# MY IMCRO

In [9]:
# Initialization


In [10]:
class MoleCule:
    def __init__(self, structure,supersequence):
        self.pe = 0
        self.ke = 0
        self.opt = 9999999
        self.num_of_hits = 0
        self.structure = copy.deepcopy(structure)
        self.supersequence = copy.deepcopy(supersequence)

    def update(self):
        """
        If this molecule has a lower energy, reset num_of_hits.
        """
        if self.pe < self.opt:
            self.opt = self.pe
            self.num_of_hits = 0
    def __str__(self):
        reVal = ''
        for i in self.structure:
            reVal+=str(i)+" "
        return reVal

In [11]:
def uni_moleReact():
    return
def inter_moleReact():
    return

In [12]:
# Iteration
i = 0
moleColl = 0.2
alpha = random.randint(10, 100)
beta = random.randint(10, 100)
while i!=1000:
    i+=1
    t = random.random()
    if t > moleColl:
        uni_moleReact()
    else :
        inter_moleReact()
    

In [20]:
def insert_symbol(src_string,inserted_string,pos):
    return ''.join(src_string[:pos] + inserted_string + src_string[pos:])
# Given set of strings and population size for SCS problem
def supersequence_generate(set_of_strings):

    '''
        Make a copy of the set_of_strings parameter for maintaining the original 
        set
    '''
    copied_set_of_strings = copy.deepcopy(set_of_strings)
    supersequence = ''.join(copied_set_of_strings.pop(random.randint(0,len(set_of_strings)-1)))

    for i in range(len(copied_set_of_strings)):
        # print("i = ",i)
        counter = 0
        for j in copied_set_of_strings[i]:
            inserted_pos = random.randint(counter,len(supersequence))
            # print("j and counter and supersequence length and inserted index",j," ",counter," ",len(supersequence)," ",inserted_pos)
            if inserted_pos == len(supersequence) or j != supersequence[inserted_pos]:
                supersequence=insert_symbol(supersequence,j,inserted_pos)
            counter = inserted_pos +1
            # print(supersequence)

    return supersequence

# supersequence_generate(['acg', 'cat', 'gtt','tgc'])
def population_generation(pop_size,set_of_strings):
    l=[]
    for i in range(pop_size):
        l.append(supersequence_generate(set_of_strings))
    return l


# Generate many supersequence with popsize = 10 and given set of strings
popsize = 10
initial_population =  population_generation(popsize,['acg', 'cat', 'gtt','tgc'])
initial_population


['gctgacgtct',
 'atcgcggatctt',
 'tgttaccgat',
 'tagccatgtt',
 'agtcgtgtact',
 'catgccagtt',
 'gtgacatgtt',
 'tgagtcatgt',
 'gttctactggc',
 'tgcatctgct']

In [21]:
def encoding_population(initial_population):
    dict = {
        'a' : 0,
        'c' : 1,
        'g' : 2,
        't' : 3
    }
    l=[]
    for i in initial_population:
        k=''
        for j in i:
            k+=str(dict[j])
        l.append(k)
    return l
encoding_population(initial_population)

['2132012313',
 '031212203133',
 '3233011203',
 '3021103233',
 '02312323013',
 '1032110233',
 '2320103233',
 '3202310323',
 '23313013221',
 '3210313213']

## Operation

### Note:
- dict = {
        'a' : 0,
        'c' : 1,
        'g' : 2,
        't' : 3
    }
- So that:
  - V_low: 0
  - V_upper: 3

In [83]:

class Operators():
    
    # OnWall Ineffective Colision
    def OnWall (self,molecule):
        '''
        Object: 
        -  This focuses on instances of molecules colliding with container walls, resulting in structural transformations.
        - A "one-difference operator" is used to make a single change in the molecule's composition to achieve this.

        Input:
        - molecule (list): the input molecule and it represent by list

        Output:
        - The method returns a new list. 
        '''
        #Initial
        m = molecule[:] 
        v_low = 0
        v_upper = 3   
        i = random.randint(0, len(molecule)-1)
        j = random.randint(v_low, v_upper)

        if (molecule[i] + j <= v_upper):    
            m[i] = molecule[i] + j
        else:
            m[i] = molecule[i] - j
        
        #Test     
        # print(i, j)
        # print(molecule)
        return m

    # Intermolecular Ineffective Colision
    def Intermolecular(self,molecule1, molecule2):
        '''
        Object: 
        -  The purpose is to introduce significant changes to enhance local search capabilities and prevent getting stuck in local optimization by promoting diversity.
        - A crossover operator is used in genetic or evolutionary algorithms for optimization. It selects two molecules from the population and uses a two-step mechanism to generate two new solutions.
        - It is a two step process: the first step is to crossover between two molecules, and the second step is to crossover inside the molecule itself

        Input:
        - molecule (list): the input molecule and it represent by list

        Output:
        - The method returns a tuple (m1, m2), where m1 and m2 are the two molecules and m1, m2 are also list. 
        '''
        #Initial 
        # Length of molecule
        length1 = len(molecule1)
        length2 = len(molecule2)
    
        #Two new molecule in first crossover
        m1 = list(range(length1)) 
        m2 = list(range(length2))
        #Two new molecule in second crossover
        m11 = [0] * length1
        m22 = [0] * length2
        
        #Random numbers x1, x2 generation
        x1 = random.randint(0, length1-2)
        x2 = random.randint(x1+1, length2-1)
    
        # Randormly choose form molecule1 or molecule2
        ## Crossover 1
        for i in range(0,length1):
            if (i<x1 or i>x2):  #if odd segments
                m1[i] = molecule1[i]
                m2[i] = molecule2[i]
            elif (i>=x1 and x2>=i): # if even segment
                m1[i] = molecule2[i]
                m2[i] = molecule1[i]
        
        # Crossover 2
        for j in range(0,length1):
            if (j < x1):  #if odd segments
                m11[length1 - x1 + j] = m1[j]
                m22[length1 - x1 + j] = m2[j]
            elif (j>=x1 and x2>=j): # if even segment
                m11[length1 - x1 - x2 + j - 1] = m1[j]
                m22[length1 - x1 - x2 + j- 1] = m2[j]
            else:
                m11[j - x2-1] = m1[j]
                m22[j - x2-1] = m2[j]
        
        #Test
        # print(molecule1)
        # print(molecule2)
        # print(x1, x2)
        # print(m1)
        # print(m2)
        
        return m11,m22
    
    # Decomposition
    def Decomposition(self, molecule):
        '''
        Object: 
        - The decomposition involves randomly selecting two numbers 'a' and 'b', and then splitting the input molecule into two new molecules, 'm1' and 'm2', based on the selected numbers. 
        - The negative number −a is used for shifting to the left a steps. 
        - The positive number j is used for shifting to the right j steps.

        Input:
        - molecule (list): the input molecule and it represent by list

        Output:
        - The method returns a tuple (m1, m2), where m1 and m2 are the two molecules and m1, m2 are also list. 
        '''
        # Step 1: Select two numbers a and b randomly
        a = random.randint(-len(molecule), 0)
        b = random.randint(0, len(molecule))
        
        # Initialize m1 and m2
        m1 = [0] * len(molecule)
        m2 = [0] * len(molecule)

        # Step 2: Decomposition of molecule into m1
        for i in range(len(molecule)):
            if i + 1 <= -a:
                m1[len(molecule) - (-a) + i] = molecule[i]
            else:
                m1[i - (-a)] = molecule[i]

        # Step 3: Decomposition of molecule into m2
        for j in range(len(molecule)):
            if j + 1 <= len(molecule) - b:
                m2[j + b] = molecule[j]
            else:
                m2[j - len(molecule) + b] = molecule[j]
                
        #Test
        # print(molecule)
        # print(a, b)
        
        return m1, m2

    # Synthesis
    def Synthesis(self, molecule1, molecule2):
        """
        Generates a new list by combining two input lists in a way that preserves the frequency of the symbols used in each input list.

        Input:
        - molecule1 (list): The first input list.
        - molecule2 (list): The second input list.

        Output:
        - The method returns a new list. 
        """
        # Generate dictionaries for the frequencies of the symbols used in each input list.
        array1 = {}
        for symbol in molecule1:
            if symbol not in array1:
                array1[symbol] = 0
            array1[symbol] += 1

        array2 = {}
        for symbol in molecule2:
            if symbol not in array2:
                array2[symbol] = 0
            array2[symbol] += 1

        # Initialize the output list.
        m_prime = []

        # Iterate over the symbols in the first input list.
        for i in range(len(molecule1)):
            symbol1 = molecule1[i]
            symbol2 = molecule2[i]

            frequency_in_array1 = array1.get(symbol1, 0)
            frequency_in_array2 = array2.get(symbol2, 0)

            if frequency_in_array1 >= frequency_in_array2:
                m_prime.append(symbol1)
            # Otherwise, add the symbol from the second input list to the output list.
            else:
                m_prime.append(symbol2)
        #test
        # print(molecule1)
        # print(molecule2)
        # print(array1)
        # print(array2)
        
        return m_prime

In [84]:
#Test
op = Operators()
m1 = [3, 1, 2, 0, 3, 0, 1, 3]
m2 = [1, 3, 0, 1, 2, 1, 3, 0]
# print(mol)
# print(mol2)
# print(m1)
# print(m2)
# print("\n")


# op.OnWall(m1)
# op.Intermolecular(m1,m2)

# op.Decomposition(m1)
op.Synthesis(m1,m2)


[3, 1, 2, 0, 3, 0, 1, 3]
[1, 3, 0, 1, 2, 1, 3, 0]
{3: 3, 1: 2, 2: 1, 0: 2}
{1: 3, 3: 2, 0: 2, 2: 1}


[3, 1, 0, 1, 3, 1, 1, 3]